In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import math
import os
import statistics
from scipy.stats import norm
import pandas as pd
import seaborn as sns

In [3]:
def getCoords(name):
    image = Image.open(name + '.jpg')
    image.save(name + '.png')

    # Load image and ensure greyscale, i.e. 'L'
    im = Image.open(name + '.png').convert('L')

    # Remap colours 
    res = im.point((lambda p: 256 if p>=200 else 0))

    res.save(name + 'BW.png')
    
    #resize image to 600x600
    image = Image.open(name + 'BW.png')
    new_image = image.resize((500, 500))
    new_image.save(name + 'BW_Resized.png')
    
    #get x, y coordinates
    image = cv2.imread(name + 'BW_Resized.png')
    white = [255,255,255] #check
    black = [0,0,0] 
    x, y = np.where(np.all(image != white, axis = 2))
    zipped = np.column_stack((x,y))

#     plt.xlim(-50, 600)
#     plt.ylim(-50, 600)
#     s = [0.5]
#     plt.scatter(x, y, s=s)
#     plt.show()
    os.remove(name + 'BW_Resized.png')
    os.remove(name + 'BW.png')
    os.remove(name + '.png')
    return x, y, image

In [4]:
def getCentre(x, y):
    # assuming middle is the median, however this is not always true if there is a large tremor
    # explore using centre of mass
    centrex = np.median(x)
    centrey = np.median(y)
    
    # assuming the centres are 300 does not work out even though this is a 600x600 pixel image, the image is not centred
    x_shifted = x - centrex
    y_shifted = y - centrey

    # note the graph is not perfectly centred which may lead to inconsistencies 
    #plt.xlim(-300, 300)
    #plt.ylim(-300, 300)
    #s = [0.5]
    #plt.scatter(x_shifted, y_shifted, s=s)
    #plt.axvline(x=0, label="x=0")
    #plt.axhline(y=0, label="y=0")
    #plt.show()
    
    return centrex, centrey

In [5]:
def calcAngleArray(centrex, centrey):
    angle_array = np.empty((500, 500), float)

    #testing method
    m = (-40 - 0)/(-20 - 0)
    angle_r = math.atan(m)
    angle_d = math.degrees(angle_r)
    #print(angle_d)

    #calculate angle between centre of spiral and all pixels in 600x600 image
    for i in range(500):
        for j in range(500):
            if (i - centrex) != 0:
                m = (j - centrey)/(i - centrex)
                angle_radians = math.atan(m)
                angle_degrees = math.degrees(angle_radians)
                angle_array[i][j] = angle_degrees

    #converting all angles to positive values
    angle_array = np.absolute(angle_array)        
    return angle_array

In [6]:
 def getSobel(image):  
    #converting to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #remove noise
    img = cv2.GaussianBlur(gray,(3,3),0)
    
    #https://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_Gradient_Sobel_Laplacian_Derivatives_Edge_Detection.php
    #5x5 kernal
    sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=9)  # x
    sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=9)  # y

    #plt.subplot(2,2,1),plt.imshow(img,cmap = 'gray')
    #plt.title('Original'), plt.xticks([]), plt.yticks([])
    #plt.subplot(2,2,2),plt.imshow(sobelx,cmap = 'gray')
    #plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
    #plt.subplot(2,2,3),plt.imshow(sobely,cmap = 'gray')
    #plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])

    #plt.show()
    return sobelx, sobely

In [7]:
def calcPhi(sobelx, sobely):    
    #https://github.com/lina-haidar/Edge-Detection-Techniques-Sobel-vs.-Canny/blob/main/sobel_with_OpenCV.py
    #compare this to plain sobelx
    abs_sobel_x64f = np.absolute(sobelx)

    #is it correct to convert to uint8?
    sobel_x_8u = np.uint8(abs_sobel_x64f)

    abs_sobel_y64f = np.absolute(sobely)
    sobel_y_8u = np.uint8(abs_sobel_y64f)

    magnitude = np.hypot(sobel_x_8u, sobel_y_8u)

    #compare arctan to arctan2
    #arctan2() method computes element-wise arc tangent of arr1/arr2 choosing the quadrant correctly.
    #note these are all positive values. Is this correct? 
    #phi is the orientation of all edges in the image
    phi = np.arctan2(sobel_y_8u , sobel_x_8u) * 180 / np.pi 
   # print(phi[100])
    return phi

In [8]:
def calcTheta(phi, angle_array):
    
    #theta is the relative orientation of all lines
    theta = phi - angle_array
    return theta


In [9]:
def getEdgeTheta(theta, phi):
    edge_only = np.empty((500, 500), float)
    spiral_only = np.empty((500, 500), float)
    theta_edge_only = theta

    #leaving black pixel angles in, compare with and without
    #doesn't make a difference
#     for i in range(len_x):
#         spiral_only[x[i]][y[i]] == 1
    
    #remove values for theta that are not on the spiral edge
    for i in range(500):
        #index, = np.where(theta_edge_only[i] >= 80)
        #print(index)
        for j in range(500):
            if phi[i][j] == 0:
                theta_edge_only[i][j] = 0
   
    return theta_edge_only

In [10]:
def convertTheta1D(theta_edge_only):   
    
    #convert 600x600 matrix to 1D for graph plotting
    theta_edge_only_1D = theta_edge_only.ravel()
    theta_edge_only_1D = np.delete(theta_edge_only_1D, np.where(theta_edge_only_1D == 0))
    theta_edge_only_1D = np.delete(theta_edge_only_1D, np.where(theta_edge_only_1D >= 70))
    return theta_edge_only_1D 

In [11]:
def renameSpiral(spiral):
    spiral_type = spiral.lower()
    renamed = ""
    
    if 'before' in spiral_type and 'r' in spiral_type:
        image = Image.open(spiral + '.jpg')
        image.save('Before Right' + '.jpg')
        renamed = 'Before Right'
    if 'before' in spiral_type and 'l' in spiral_type:
        image = Image.open(spiral + '.jpg')
        image.save('Before Left' + '.jpg')
        renamed = 'Before Left'
    if '1' in spiral_type and 'y' in spiral_type and 'r' in spiral_type:
        image = Image.open(spiral + '.jpg')
        image.save('1Y Right' + '.jpg')   
        renamed = '1Y Right'
    if '1' in spiral_type and 'y' in spiral_type and 'l' in spiral_type:
        image = Image.open(spiral + '.jpg')
        image.save('1Y Left' + '.jpg')  
        renamed = '1Y Left'
   
    return renamed
    

In [12]:
#directory of folder with spirals
tremors = os.listdir('C:\\Users\\robyn\\OneDrive\\Documents\\4th Year\\Investigation Project\\AngleTreatmentAnalysis\\Spirals\\')
improvedAfter1Y = []
drawing_A_right = []
file = ""
#continue while there are spiral images in the file directory
for tremor in tremors:
    patients = os.listdir('C:\\Users\\robyn\\OneDrive\\Documents\\4th Year\\Investigation Project\\AngleTreatmentAnalysis\\Spirals\\'+tremor+'\\')
    for patient in patients:
        sdArray = np.zeros((5), float)
        sdArray[0] = int(patient.removeprefix('#'))
        drawings = os.listdir('C:\\Users\\robyn\\OneDrive\\Documents\\4th Year\\Investigation Project\\AngleTreatmentAnalysis\\Spirals\\'+tremor+'\\'+patient+'\\')
        for drawing in drawings: 
            #Just Drawing A for now
            if drawing == "DrawingA":
                spirals = os.listdir('C:\\Users\\robyn\\OneDrive\\Documents\\4th Year\\Investigation Project\\AngleTreatmentAnalysis\\Spirals\\'+tremor+'\\'+patient+'\\'+drawing+'\\')

                for spiral in spirals: 
                    if '.jpg' in spiral:
                            spiral = spiral.removesuffix('.jpg')
                            spiral_type = spiral.lower()
                            #note this could be blank if spiral is not named normally
                            #spiral = renameSpiral(spiral)                          
                            
                            file = 'Spirals'+'\\'+tremor+'\\'+patient+'\\'+drawing+'\\'
                            #print(spiral)
                            
                            ######################
                            x, y, image = getCoords(file+spiral)
                            centrex, centrey = getCentre(x, y)
                            angle_array = calcAngleArray(centrex, centrey)
                            sobelx, sobely = getSobel(image)
                            phi = calcPhi(sobelx, sobely)
                            theta = calcTheta(phi, angle_array)
                            theta_edge_only = getEdgeTheta(theta, phi)
                            theta_edge_only_1D = convertTheta1D(theta_edge_only) 

                            y, binEdges = np.histogram(theta_edge_only_1D, bins = [ -90, -80, -70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80])
                            bincenters = 0.5 * (binEdges[1:] + binEdges[:-1])

                            sd = np.std(theta_edge_only_1D)
                            mean = np.mean(theta_edge_only_1D)
                            #print(sd)
                            #####################
                            
                            if ('before' in spiral_type) and (('rt' in spiral_type) or ('right' in spiral_type)):
                                sdArray[1] = sd
                            if ('1' in spiral_type) and ('y' in spiral_type) and ('r' in spiral_type):
                                sdArray[2] = sd
                            if ('2' in spiral_type) and ('y' in spiral_type) and ('r' in spiral_type):
                                sdArray[3] = sd
                            if ('3' in spiral_type) and ('y' in spiral_type) and ('r' in spiral_type):
                                sdArray[4] = sd

        print(sdArray)
        drawing_A_right.append(sdArray)
                
            


[ 1.         33.29364457 39.55327937 32.63712048 32.41440357]
[10.          0.         33.19206343 33.26433481 33.19206343]
[11.         32.74203412  0.          0.          0.        ]
[12.         33.60765437 32.82735907 32.82735907  0.        ]
[ 2.          0.         32.69720586 32.80544481  0.        ]
[ 5.         33.33535683 32.62624205 32.83103494 32.76272042]
[ 6.         32.80961414 32.83287007 32.90954326  0.        ]
[ 9.         32.68277491 32.78049023  0.         33.11524612]
[16.  0.  0.  0.  0.]
[18.         32.76234707 31.47517934 31.47517934 32.6594638 ]
[ 3.          0.         32.70205711 32.64648033 32.64385053]
[ 4.         32.88871544 32.56516239 32.85408658 32.50780398]
[ 7.          0.         32.75679291 32.8674971  32.5650102 ]
[ 8.         32.55257583 32.89766231  0.          0.        ]


In [13]:
print(drawing_A_right)
drawing_A_right_table = pd.DataFrame(drawing_A_right)
drawing_A_right_table.columns =['Patient', 'Before', '1 Year', '2 Years', '3 Years']
drawing_A_right_table.sort_values(by=['Patient'])

print("Drawing A Right Table")
print(drawing_A_right_table)



[array([ 1.        , 33.29364457, 39.55327937, 32.63712048, 32.41440357]), array([10.        ,  0.        , 33.19206343, 33.26433481, 33.19206343]), array([11.        , 32.74203412,  0.        ,  0.        ,  0.        ]), array([12.        , 33.60765437, 32.82735907, 32.82735907,  0.        ]), array([ 2.        ,  0.        , 32.69720586, 32.80544481,  0.        ]), array([ 5.        , 33.33535683, 32.62624205, 32.83103494, 32.76272042]), array([ 6.        , 32.80961414, 32.83287007, 32.90954326,  0.        ]), array([ 9.        , 32.68277491, 32.78049023,  0.        , 33.11524612]), array([16.,  0.,  0.,  0.,  0.]), array([18.        , 32.76234707, 31.47517934, 31.47517934, 32.6594638 ]), array([ 3.        ,  0.        , 32.70205711, 32.64648033, 32.64385053]), array([ 4.        , 32.88871544, 32.56516239, 32.85408658, 32.50780398]), array([ 7.        ,  0.        , 32.75679291, 32.8674971 , 32.5650102 ]), array([ 8.        , 32.55257583, 32.89766231,  0.        ,  0.        ])]
Dra